# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb22489ffa0>
├── 'a' --> tensor([[-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445]])
└── 'x' --> <FastTreeValue 0x7fb22489ff70>
    └── 'c' --> tensor([[ 0.7673,  0.1922, -0.5657, -0.4330],
                        [-0.2190,  0.0874, -0.0133, -1.0593],
                        [ 0.9928, -0.0634,  0.9964,  2.4555]])

In [4]:
t.a

tensor([[-0.0320, -1.0748,  0.0411],
        [-0.3698, -0.3416,  0.1445]])

In [5]:
%timeit t.a

64.1 ns ± 0.0374 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb22489ffa0>
├── 'a' --> tensor([[ 0.5364, -0.1742, -0.1876],
│                   [ 0.9007, -0.9891, -1.0720]])
└── 'x' --> <FastTreeValue 0x7fb22489ff70>
    └── 'c' --> tensor([[ 0.7673,  0.1922, -0.5657, -0.4330],
                        [-0.2190,  0.0874, -0.0133, -1.0593],
                        [ 0.9928, -0.0634,  0.9964,  2.4555]])

In [7]:
%timeit t.a = new_value

68.8 ns ± 0.0898 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0320, -1.0748,  0.0411],
               [-0.3698, -0.3416,  0.1445]]),
    x: Batch(
           c: tensor([[ 0.7673,  0.1922, -0.5657, -0.4330],
                      [-0.2190,  0.0874, -0.0133, -1.0593],
                      [ 0.9928, -0.0634,  0.9964,  2.4555]]),
       ),
)

In [10]:
b.a

tensor([[-0.0320, -1.0748,  0.0411],
        [-0.3698, -0.3416,  0.1445]])

In [11]:
%timeit b.a

59 ns ± 0.0411 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4324, -1.4320,  0.1500],
               [-0.4003, -1.3175, -0.4529]]),
    x: Batch(
           c: tensor([[ 0.7673,  0.1922, -0.5657, -0.4330],
                      [-0.2190,  0.0874, -0.0133, -1.0593],
                      [ 0.9928, -0.0634,  0.9964,  2.4555]]),
       ),
)

In [13]:
%timeit b.a = new_value

504 ns ± 0.252 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

815 ns ± 3.38 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 364 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 498 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb22490c550>
├── 'a' --> tensor([[[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]],
│           
│                   [[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]],
│           
│                   [[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]],
│           
│                   [[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]],
│           
│                   [[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]],
│           
│                   [[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]],
│           
│                   [[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]],
│           
│                   [[-0.0320, -1.0748,  0.0411],
│                    [-0.3698, -0.3416,  0.1445]]])
└── 'x' --> <FastTreeValue 0x7fb170132b20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb22490c100>
├── 'a' --> tensor([[-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445],
│                   [-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445],
│                   [-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445],
│                   [-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445],
│                   [-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445],
│                   [-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445],
│                   [-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445],
│                   [-0.0320, -1.0748,  0.0411],
│                   [-0.3698, -0.3416,  0.1445]])
└── 'x' --> <FastTreeValue 0x7fb22490ce50>
    └── 'c' --> tensor([[ 0.7673,  0.1922, -0.5657, -0.4330],
                        [-0.2190,  0.0874, -0.0133, -1.0593],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 36.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.4 µs ± 68.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7673,  0.1922, -0.5657, -0.4330],
                       [-0.2190,  0.0874, -0.0133, -1.0593],
                       [ 0.9928, -0.0634,  0.9964,  2.4555]],
              
                      [[ 0.7673,  0.1922, -0.5657, -0.4330],
                       [-0.2190,  0.0874, -0.0133, -1.0593],
                       [ 0.9928, -0.0634,  0.9964,  2.4555]],
              
                      [[ 0.7673,  0.1922, -0.5657, -0.4330],
                       [-0.2190,  0.0874, -0.0133, -1.0593],
                       [ 0.9928, -0.0634,  0.9964,  2.4555]],
              
                      [[ 0.7673,  0.1922, -0.5657, -0.4330],
                       [-0.2190,  0.0874, -0.0133, -1.0593],
                       [ 0.9928, -0.0634,  0.9964,  2.4555]],
              
                      [[ 0.7673,  0.1922, -0.5657, -0.4330],
                       [-0.2190,  0.0874, -0.0133, -1.0593],
                       [ 0.9928, -0.0634,  0.9964,  2.4555]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7673,  0.1922, -0.5657, -0.4330],
                      [-0.2190,  0.0874, -0.0133, -1.0593],
                      [ 0.9928, -0.0634,  0.9964,  2.4555],
                      [ 0.7673,  0.1922, -0.5657, -0.4330],
                      [-0.2190,  0.0874, -0.0133, -1.0593],
                      [ 0.9928, -0.0634,  0.9964,  2.4555],
                      [ 0.7673,  0.1922, -0.5657, -0.4330],
                      [-0.2190,  0.0874, -0.0133, -1.0593],
                      [ 0.9928, -0.0634,  0.9964,  2.4555],
                      [ 0.7673,  0.1922, -0.5657, -0.4330],
                      [-0.2190,  0.0874, -0.0133, -1.0593],
                      [ 0.9928, -0.0634,  0.9964,  2.4555],
                      [ 0.7673,  0.1922, -0.5657, -0.4330],
                      [-0.2190,  0.0874, -0.0133, -1.0593],
                      [ 0.9928, -0.0634,  0.9964,  2.4555],
                      [ 0.7673,  0.1922, -0.5657, -0.4330],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 2.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
